In [ ]:
import numpy as np
import pymc3 as pm
import ray
from ray import tune

#Def variables
x = np.random.randn(100, 2)
y = np.random.randn(100)

def train_bart(config):
    # Modelo BART
    with pm.Model() as bart_model:
        # (Definición del modelo BART)

        # Número de iteraciones de MCMC
        num_samples = config.get("num_samples", 1000)

        # Muestreo MCMC
        trace = pm.sample(num_samples)

        # Guardar los resultados o métricas que quieras registrar, como el error de prueba, etc.

        # Devolver la métrica que quieres maximizar o minimizar durante la búsqueda de hiperparámetros
        return -np.mean(trace['y_obs'])

if __name__ == "__main__":
    # Inicializar Ray (debes hacer esto antes de llamar a tune.run())
    ray.init(ignore_reinit_error=True)

    # Configuración de hiperparámetros a ajustar
    config = {
        "num_samples": tune.grid_search([1000, 2000, 3000]), # Puedes ajustar otros hiperparámetros también
        # Agrega otros hiperparámetros para ajustar aquí...
    }

    # Lanzar la búsqueda de hiperparámetros en paralelo
    result = tune.run(train_bart, config=config, num_samples=3, resources_per_trial={"cpu": 1})

    # Obtener los mejores hiperparámetros encontrados y sus resultados
    best_config = result.get_best_config(metric="neg_mean")  # Metric to maximize/minimize
    best_result = result.get_best_trial(metric="neg_mean", mode="max")  # Metric to maximize

    print("Mejores hiperparámetros:", best_config)
    print("Mejor resultado:", best_result)


In [ ]:
#Como imprimir los resultados en la consola para ver los hiperparámetros y las métricas de rendimiento de cada ejecución

if __name__ == "__main__":

    # Lanzar la búsqueda de hiperparámetros en paralelo
    result = tune.run(train_bart, config=config, num_samples=3, resources_per_trial={"cpu": 1})

    # Obtener los mejores hiperparámetros encontrados y sus resultados
    best_config = result.get_best_config(metric="neg_mean")  # Metric to maximize/minimize
    best_result = result.get_best_trial(metric="neg_mean", mode="max")  # Metric to maximize

    # Imprimir todos los resultados
    print("Resultados:")
    for trial in result.trials:
        print("Hiperparámetros:", trial.config)
        print("Métrica de rendimiento:", trial.last_result)
        print("-----")

    print("Mejores hiperparámetros:", best_config)
    print("Mejor resultado:", best_result)

In [ ]:
#Para ver los resultados de cada ejecución de Ray Tune

if __name__ == "__main__":

    # Lanzar la búsqueda de hiperparámetros en paralelo
    result = tune.run(train_bart, config=config, num_samples=3, resources_per_trial={"cpu": 1})

    # Obtener los resultados en una lista
    results_list = []
    for trial in result.trials:
        results_list.append((trial.config, trial.last_result))

    # Imprimir todos los resultados
    print("Resultados:")
    for config, result in results_list:
        print("Hiperparámetros:", config)
        print("Métrica de rendimiento:", result)
        print("-----")

    # Obtener los mejores hiperparámetros encontrados y sus resultados
    best_config = result.get_best_config(metric="neg_mean")  # Metric to maximize/minimize
    best_result = result.get_best_trial(metric="neg_mean", mode="max")  # Metric to maximize

    print("Mejores hiperparámetros:", best_config)
    print("Mejor resultado:", best_result)


In [ ]:
#Para visualizar resultados graficamente con Matplotlib

import matplotlib.pyplot as plt

if __name__ == "__main__":

    # Lanzar la búsqueda de hiperparámetros en paralelo
    result = tune.run(train_bart, config=config, num_samples=3, resources_per_trial={"cpu": 1})

    # Obtener los resultados en una lista
    results_list = []
    for trial in result.trials:
        results_list.append((trial.config, trial.last_result))

    # Imprimir todos los resultados
    print("Resultados:")
    for config, result in results_list:
        print("Hiperparámetros:", config)
        print("Métrica de rendimiento:", result)
        print("-----")

    # Obtener los mejores hiperparámetros encontrados y sus resultados
    best_config = result.get_best_config(metric="neg_mean")  # Metric to maximize/minimize
    best_result = result.get_best_trial(metric="neg_mean", mode="max")  # Metric to maximize

    print("Mejores hiperparámetros:", best_config)
    print("Mejor resultado:", best_result)

    # Graficar la métrica de rendimiento en función de "num_samples"
    num_samples_list = [config["num_samples"] for config, _ in results_list]
    metric_values = [-result["neg_mean"] for _, result in results_list]  # Tomamos el negativo para maximizar

    plt.plot(num_samples_list, metric_values, marker='o')
    plt.xlabel("Número de muestras de MCMC")
    plt.ylabel("Error medio cuadrático negativo")
    plt.title("Evaluación del rendimiento en función de 'num_samples'")
    plt.grid(True)
    plt.show()
